In [ ]:
import librosa
import numpy as np
from IPython.display import Audio
from Preprocess.pythonProject.audio_preprocess import *

In [8]:
def load_and_trim(path, sr=16000):  # memmap对象，它允许将大文件分成小段进行读写，而不是一次性将整个数组读入内存。
    min_length = 1 * sr  # 分片最小长度
    slice_length = 3 * sr  # 分片最大长度
    audio = np.memmap(path, dtype='h', mode='r')  # 使用函数np.memmap并传入一个文件路径、数据类型、形状以及文件模式
    # audio = audio[2000:-2000]                    #b = a[i:j] 表示复制a[i]到a[j-1]，为负数时表示倒数几个，以生成新的list对象
    audio = audio.astype(np.float32)  # astype()：类型转换为float32
    energy = librosa.feature.rms(audio)  # 调用librosa中的rmse直接对音频每个帧进行计算得到均方根能量
    frames = np.nonzero(energy >= np.max(energy) / 5)  # 返回非0值的地址
    indices = librosa.core.frames_to_samples(frames)[1]  # librosa.core.frames_to_samples():将帧索引转换为音频样本索引
    audio = audio[indices[0]:indices[-1]] if indices.size else audio[0:0]  # 三目运算，audio中存放去除头尾的音频信号

    slices = []
    for i in range(0, audio.shape[0], slice_length):  # 对长语音片段分片，shapr数组大小
        s = audio[i: i + slice_length]
        if s.shape[0] >= min_length:
            slices.append(s)  # append在列表尾插入元素，slices中存放分片后的音频信号
    
    return audio, slices